# **Proyek Sistem Rekomendasi: Rekomendasi Film 1996 - 2018**

## **Import Library**

## **Loading Dataset**

## **Exploratory Data Analysis (EDA)**

## **Data Preparation**

## **Building Model**

## **Evaluating Model**

## **Hyperparameter Tuning**

## **Evaluating Model after Tuning**